# Demonstrate loading and using ERNIE4us

## Demo loading the ERNIE2 model and preparing inputs

In [ ]:
import numpy as np
import dataclasses
import os
import sys
import logging
import tensorflow as tf
from ernie4us import *

In [ ]:
tf.__version__

In [ ]:
ernie_model = ERNIE_LARGE_EN
ernie_path = 'model_artifacts/%s' % ernie_model

In [ ]:
ernie_input_builder, ernie_tf_inputs, ernie_tf_outputs = load_ernie_model(ernie_model,
    "./model_artifacts")

In [ ]:
session = tf.compat.v1.Session(graph=ernie_tf_inputs.token_ids.graph)

In [ ]:
for i, op in enumerate(session.graph.get_operations()):
    if op.type == 'Const':
        tensor = session.graph.get_tensor_by_name(op.name + ':0')
        print(i, '\t', tensor, tensor.op.type)

In [ ]:
session.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

In [ ]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

In [ ]:
import os, sys

# NOTE(paddle-dev): All of these flags should be
# set before `import paddle`. Otherwise, it would
# not take any effect.
os.environ['FLAGS_eager_delete_tensor_gb'] = '0'  # enable gc

import paddle.fluid as fluid

try:
    from model.ernie import ErnieConfig
except ModuleNotFoundError:
    sys.path.insert(0, os.path.join('third_party', 'ERNIE'))

from model.ernie import ErnieConfig
from utils.init import init_pretraining_params, init_checkpoint
from model.ernie import ErnieModel

max_seq_len=512

def create_model(ernie_config):
    input_names = ("src_ids", "sent_ids", "pos_ids", "task_ids", "input_mask")
    shapes=[[-1, max_seq_len, 1], [-1, max_seq_len, 1],
            [-1, max_seq_len, 1], [-1, max_seq_len, 1],
            [-1, max_seq_len, 1]]
    dtypes=[
        'int64', 'int64', 'int64', 'int64', 'float32'
    ]

    inputs = [fluid.data(name, shape, dtype=dtype) for name, shape, dtype in zip(input_names, shapes, dtypes)]
    (src_ids, sent_ids, pos_ids, task_ids, input_mask) = inputs

    ernie = ErnieModel(
        src_ids=src_ids,
        position_ids=pos_ids,
        sentence_ids=sent_ids,
        task_ids=task_ids,
        input_mask=input_mask,
        config=ernie_config,
        use_fp16=False)

    seq_out = ernie.get_sequence_output()
    cls_feats = ernie.get_pooled_output()
    # dummy layers to name the latent layers. the save_inf_model produce uncomprehensible names
    # like 'save_infer_model/scale_1'
    seq_out = fluid.layers.scale(seq_out, scale=1.0, name='ernie_sequence_latent')
    cls_feats = fluid.layers.scale(cls_feats, scale=1.0, name='ernie_classification')

    for i, inp in enumerate(inputs):
        print(f'input[{i}]:', inp.name, inp.shape, inp.dtype)
    print('sequence_output  :', seq_out.name, seq_out.shape, seq_out.dtype)
    print('classifier_output:', cls_feats.name, cls_feats.shape, cls_feats.dtype)
    return inputs, [seq_out, cls_feats]


pretraining_params_path = f'{ernie_path}/paddle/params'
ernie_config_path = f'{ernie_path}/paddle/ernie_config.json'
ernie_vocab_path = f'{ernie_path}/paddle/vocab.txt'
ernie_config = ErnieConfig(ernie_config_path)
# Fix missing use_task_id
ernie_config._config_dict['use_task_id'] = True
ernie_config.print_config()

place = fluid.CPUPlace()
exe = fluid.Executor(place)


startup_prog = fluid.Program()
predict_prog = fluid.Program()

with fluid.program_guard(predict_prog, startup_prog):
    with fluid.unique_name.guard():
        inputs, ernie_latent = create_model(ernie_config=ernie_config)
        predict_prog = predict_prog.clone(for_test=True)
with fluid.program_guard(predict_prog, startup_prog):
    init_pretraining_params(
        exe,
        pretraining_params_path,
        main_program=startup_prog,
        use_fp16=False)

In [ ]:
output_paddle_names = []
with fluid.program_guard(predict_prog, startup_prog):
    block = predict_prog.global_block()
    for op in block.ops:
        print(op.type, op.output(op.output_names[0]), len(op.output_names))
        for on in op.output_names:
            output_paddle_names.extend(op.output(on))
output_tf_names = [n.replace('@', '_') for n in output_paddle_names]
print(len(output_paddle_names), len(output_tf_names))

In [ ]:
type(ernie_latent[1])

In [ ]:
def zip_tf_feed_dict(tensors, values, is_training=False):
    feed_dict = {getattr(tensors, field.name): values[field.name] if isinstance(values, dict) 
                 else getattr(values, field.name) for field in dataclasses.fields(tensors)}
    for tensor in feed_dict.keys():
        assert isinstance(tensor, tf.Tensor), f"key is of type {tensor}"
    return feed_dict

with fluid.program_guard(predict_prog, startup_prog):
    text = "Apartment was okay still some fixtures to be done in bathroom. Hallways walls need to be cleaned and painted. narrow hallways. and hallway rugs need to be cleaned."

    paddle_input_builder = Ernie2InputBuilder(ernie_vocab_path, do_lower_case=True, max_seq_len=max_seq_len)
    paddle_inputs = paddle_input_builder.build(text)
    paddle_inputs = {
        "src_ids": np.reshape(paddle_inputs.token_ids, (1, -1, 1)),
        "sent_ids": np.reshape(paddle_inputs.sentence_ids, (1, -1, 1)), 
        "pos_ids": np.reshape(paddle_inputs.position_ids, (1, -1, 1)), 
        "task_ids": np.reshape(paddle_inputs.task_ids, (1, -1, 1)),
        "input_mask": np.reshape(paddle_inputs.input_mask, (1, -1, 1))
    }
    paddle_latent = exe.run(fluid.default_main_program(),
          feed=paddle_inputs,
          fetch_list=ernie_latent[1])[0]
    tf_inputs = ernie_input_builder.build(text)
    tf_latent = session.run(ernie_tf_outputs.classification_features, 
                            feed_dict=zip_tf_feed_dict(ernie_tf_inputs, tf_inputs))


In [ ]:
cos_dist = np.dot(paddle_latent, np.transpose(tf_latent)) \
    / (np.linalg.norm(paddle_latent) * np.linalg.norm(tf_latent))
cos_dist = np.reshape(cos_dist, [])
print('paddle_latent>', paddle_latent.shape, 'tf_latent>', tf_latent.shape,
      float(cos_dist))
assert cos_dist > 0.9999